In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
from io import StringIO
import sklearn as skn
from scipy import stats
import pandas_datareader.data as pdr
import riskfolio as rp
from scipy.stats import multivariate_normal


In [2]:
def loadStooqData(ticker: str,start, frequency='d'):
    url = f'https://stooq.pl/q/d/l/?s={ticker}&i={frequency}'
    response = requests.get(url)
    data = pd.read_csv(StringIO(response.text))
    data.set_index('Data', inplace=True)
    data = data[start:]
    return data['Zamkniecie']

def loadYahooData(ticker, start, frequency = '1d'):
    ticker  = yf.Ticker(ticker)
    hist = ticker.history(start=start,interval=frequency)
    price = hist['Close']
    return price

def sharp(returns):
    return returns.mean()/returns.std()

def assets_performance(returns: pd.DataFrame):
    return returns.agg(['mean', 'std', 'median', 'skew', 'kurtosis', sharp])

def beta(X, Y):

    X_cov = X.cov()

    XY = np.append(X,np.expand_dims(Y,axis=1),axis=0)
    XY_cov = np.cov(XY)

    #XY_sub = XY_cov.iloc[-1,0:2]

    #inv = np.linalg.inv(X_cov)
    #B = inv @ XY_sub
    return  XY_cov

In [212]:
yahoo_tickers = ['ACWI', 'CORP.L', 'IHYU.L']

stooq_tickers = ['XAUPLN', '^TBSP', 'MWIG40', 'USDPLN', 'EURPLN', 'PLOPLN3M']

#na przyszłość EAFA,EEM,ACWI



In [213]:
base_prices = pd.DataFrame({})
start = '2008-01-01'

for ticker in stooq_tickers:
    base_prices[ticker] = loadStooqData(ticker,start)

for ticker in yahoo_tickers:
    base_prices[ticker] = loadYahooData(ticker,start)

base_prices.index = pd.DatetimeIndex(base_prices.index)


In [214]:
#get monthly data
monthly_base_prices = base_prices.resample('m').last()

In [215]:
monthly_base_returns = np.log(monthly_base_prices/monthly_base_prices.shift(1))
assets_performance(monthly_base_returns)

,XAUPLN,^TBSP,MWIG40,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L,IHYU.L
mean,0.006663,0.003241,0.002371,0.002822,0.001150,0.000896,0.005091,-0.001285,-0.000573
std,0.053793,0.012802,0.060642,0.040162,0.021733,0.149128,0.050699,0.019803,0.023279
median,0.003995,0.003729,0.006322,0.000648,-0.000957,0.000000,0.011982,0.001964,0.001948
skew,0.601099,0.207804,-0.969706,0.787940,0.666798,2.912263,-0.751152,-0.687892,-0.210055
kurtosis,1.900035,5.204837,5.534715,1.994325,2.582320,38.493551,1.480117,2.497974,2.393593
sharp,0.123869,0.253151,0.039093,0.070267,0.052894,0.006005,0.100411,-0.064872,-0.024626


In [216]:
monthly_base_returns.corr()

,XAUPLN,^TBSP,MWIG40,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L,IHYU.L
XAUPLN,1.000000,0.086375,-0.263496,0.486853,0.525780,-0.020658,-0.382789,0.034327,0.008703
^TBSP,0.086375,1.000000,0.157029,-0.202445,-0.103132,-0.383582,0.232811,0.545653,0.263614
MWIG40,-0.263496,0.157029,1.000000,-0.565152,-0.537998,-0.038508,0.772405,0.543186,0.549005
USDPLN,0.486853,-0.202445,-0.565152,1.000000,0.751591,0.018887,-0.715089,-0.596361,-0.410698
EURPLN,0.525780,-0.103132,-0.537998,0.751591,1.000000,-0.028417,-0.568147,-0.422867,-0.400413
PLOPLN3M,-0.020658,-0.383582,-0.038508,0.018887,-0.028417,1.000000,-0.021718,-0.117016,-0.089868
ACWI,-0.382789,0.232811,0.772405,-0.715089,-0.568147,-0.021718,1.000000,0.692023,0.710521
CORP.L,0.034327,0.545653,0.543186,-0.596361,-0.422867,-0.117016,0.692023,1.000000,0.599064
IHYU.L,0.008703,0.263614,0.549005,-0.410698,-0.400413,-0.089868,0.710521,0.599064,1.000000


In [217]:
#calculate pln returns
pln_prices = monthly_base_prices
pln_prices['ACWI'] = monthly_base_prices['ACWI'] * monthly_base_prices['USDPLN']
pln_prices['CORP.L'] = monthly_base_prices['CORP.L'] * monthly_base_prices['USDPLN']
pln_returns = np.log(pln_prices/pln_prices.shift(1))
pln_returns['PLOPLN3M'] = pln_prices['PLOPLN3M'] / 100 / 12
assets_performance(pln_returns)

,XAUPLN,^TBSP,MWIG40,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L,IHYU.L
mean,0.006663,0.003241,0.002371,0.002822,0.001150,0.002665,0.008419,0.000661,-0.000573
std,0.053793,0.012802,0.060642,0.040162,0.021733,0.001677,0.035646,0.023706,0.023279
median,0.003995,0.003729,0.006322,0.000648,-0.000957,0.002242,0.008942,0.001258,0.001948
skew,0.601099,0.207804,-0.969706,0.787940,0.666798,0.476330,-0.265478,0.238006,-0.210055
kurtosis,1.900035,5.204837,5.534715,1.994325,2.582320,-0.820378,0.649281,-0.364492,2.393593
sharp,0.123869,0.253151,0.039093,0.070267,0.052894,1.589370,0.236171,0.027888,-0.024626


In [218]:
pln_returns.corr()

,XAUPLN,^TBSP,MWIG40,USDPLN,EURPLN,PLOPLN3M,ACWI,CORP.L,IHYU.L
XAUPLN,1.000000,0.086375,-0.263496,0.486853,0.525780,0.021514,-0.000549,0.487472,0.008703
^TBSP,0.086375,1.000000,0.157029,-0.202445,-0.103132,0.130031,0.093793,0.162401,0.263614
MWIG40,-0.263496,0.157029,1.000000,-0.565152,-0.537998,-0.157388,0.457051,-0.116115,0.549005
USDPLN,0.486853,-0.202445,-0.565152,1.000000,0.751591,0.040317,0.107197,0.741854,-0.410698
EURPLN,0.525780,-0.103132,-0.537998,0.751591,1.000000,0.016249,0.036754,0.553214,-0.400413
PLOPLN3M,0.021514,0.130031,-0.157388,0.040317,0.016249,1.000000,-0.153316,-0.149723,-0.017412
ACWI,-0.000549,0.093793,0.457051,0.107197,0.036754,-0.153316,1.000000,0.484284,0.491908
CORP.L,0.487472,0.162401,-0.116115,0.741854,0.553214,-0.149723,0.484284,1.000000,0.060221
IHYU.L,0.008703,0.263614,0.549005,-0.410698,-0.400413,-0.017412,0.491908,0.060221,1.000000


In [219]:
pln_returns.mean() * 12

XAUPLN      0.079959
^TBSP       0.038889
MWIG40      0.028448
USDPLN      0.033865
EURPLN      0.013795
PLOPLN3M    0.031976
ACWI        0.101022
CORP.L      0.007933
IHYU.L     -0.006879
dtype: float64

In [186]:
expected_returns = pd.Series({
'ACWI': 0.085/12,
'^TBSP': 0.035/12,
'XAUPLN': 0.065/12,
'MWIG40TR': 0.080/12,
'IHYU.L': 0.038/12,
'PLOPLN3M': 0.03/12
})


In [257]:
assets = ['ACWI', '^TBSP', 'XAUPLN', 'MWIG40', 'PLOPLN3M']
#
cov_matrix = pln_returns[assets].cov()

* load data
* describe data in local currency
* transform data do pln
* describe data in pln

In [258]:
portfolio = rp.Portfolio(pd.DataFrame(pln_returns[assets]))
#pln_returns[assets]

portfolio.mu = expected_returns
portfolio.cov = cov_matrix

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

ValueError: Incompatible dimensions (5, 5) (6, 1)

In [259]:
w = portfolio.optimization(model=model, rm=rm, obj='MaxRet', rf=rf, l=l, hist=hist)

display(np.round(w.T,2))

ValueError: Incompatible dimensions (5, 5) (6, 1)

In [260]:
def frontier_performance(frontier,mu,cov,rf = 0.0):
    result = frontier.copy()
    result['Returns'] = np.round(frontier.to_numpy() @ mu *12,4)
    result['Std. deviations'] = frontier.apply(lambda x: np.round(np.sqrt(x.to_numpy().T @ cov @ x.to_numpy())* np.sqrt(12),4),axis=1)
    result['SR'] = np.round((result['Returns']-rf) / result['Std. deviations'],2)
    return result


In [261]:
def shrinkedCovariance(returns: pd.DataFrame, w: int):
    std = np.diag(returns.std())
    corr = returns.corr('pearson')
    shrinked_corr = (1-w)*corr + np.ones_like(corr)*w
    result = std @ shrinked_corr @ std
    return result

shrin_param = 0.75

In [262]:
points = 10
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10 = np.round(frontier.T,2)

ValueError: Incompatible dimensions (5, 5) (6, 1)

In [ ]:
frontier_performance(frontier_10, expected_returns,cov_matrix,0.0298)

,ACWI,^TBSP,XAUPLN,MWIG40TR,IHYU.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.01,0.00,0.00,0.00,0.0,0.99,0.0306,0.0057,0.14
1,0.11,0.02,0.04,0.04,0.0,0.79,0.0396,0.0186,0.53
2,0.19,0.04,0.08,0.07,0.0,0.63,0.0472,0.0328,0.53
3,0.26,0.06,0.11,0.09,0.0,0.48,0.0530,0.0444,0.52
4,0.34,0.08,0.14,0.12,0.0,0.32,0.0600,0.0582,0.52
5,0.41,0.09,0.17,0.15,0.0,0.17,0.0662,0.0708,0.51
6,0.49,0.11,0.20,0.18,0.0,0.02,0.0735,0.0846,0.52
7,0.60,0.00,0.21,0.20,0.0,0.00,0.0806,0.0983,0.52
8,0.80,0.00,0.07,0.13,0.0,0.00,0.0830,0.1099,0.48
9,1.00,0.00,0.00,0.00,0.0,0.00,0.0850,0.1235,0.45


In [ ]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10_sh = np.round(frontier.T,2)

In [ ]:
frontier_performance(frontier_10_sh,mu,shrinked_cov,0.0298)


,ACWI,^TBSP,XAUPLN,MWIG40TR,IHYU.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.0,0.0,0.00,0.0,0.0,1.00,0.0300,0.0058,0.03
1,0.0,0.0,0.12,0.0,0.0,0.88,0.1224,0.0264,3.51
2,0.0,0.0,0.23,0.0,0.0,0.77,0.2071,0.0463,3.83
3,0.0,0.0,0.34,0.0,0.0,0.66,0.2918,0.0663,3.95
4,0.0,0.0,0.45,0.0,0.0,0.55,0.3765,0.0863,4.02
5,0.0,0.0,0.56,0.0,0.0,0.44,0.4612,0.1063,4.06
6,0.0,0.0,0.67,0.0,0.0,0.33,0.5459,0.1263,4.09
7,0.0,0.0,0.78,0.0,0.0,0.22,0.6306,0.1463,4.11
8,0.0,0.0,0.89,0.0,0.0,0.11,0.7153,0.1664,4.12
9,0.0,0.0,1.00,0.0,0.0,0.00,0.8000,0.1864,4.13


In [ ]:
mu = expected_returns
cov = pln_returns[0:60][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5 = np.round(frontier.T,2)

In [ ]:
frontier_performance(frontier_1_5,mu,cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,IHYU.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.01,0.0,0.00,0.00,0.0,0.99,0.0306,0.0025,12.24
1,0.02,0.0,0.13,0.11,0.0,0.74,0.1367,0.0289,4.73
2,0.03,0.0,0.26,0.20,0.0,0.51,0.2418,0.0574,4.21
3,0.05,0.0,0.38,0.30,0.0,0.28,0.3406,0.0841,4.05
4,0.06,0.0,0.50,0.39,0.0,0.04,0.4375,0.1106,3.96
5,0.00,0.0,0.63,0.37,0.0,0.00,0.5336,0.1387,3.85
6,0.00,0.0,0.73,0.27,0.0,0.00,0.6056,0.1646,3.68
7,0.00,0.0,0.83,0.17,0.0,0.00,0.6776,0.1935,3.50
8,0.00,0.0,0.91,0.09,0.0,0.00,0.7352,0.2179,3.37
9,0.00,0.0,1.00,0.00,0.0,0.00,0.8000,0.2464,3.25


In [ ]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[0:60][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5_sh = np.round(frontier.T,2)

In [ ]:
frontier_performance(frontier_1_5_sh,mu,shrinked_cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,IHYU.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.0,0.0,0.00,0.0,0.0,1.00,0.0300,0.0025,12.00
1,0.0,0.0,0.11,0.0,0.0,0.89,0.1147,0.0288,3.98
2,0.0,0.0,0.22,0.0,0.0,0.78,0.1994,0.0557,3.58
3,0.0,0.0,0.34,0.0,0.0,0.66,0.2918,0.0850,3.43
4,0.0,0.0,0.45,0.0,0.0,0.55,0.3765,0.1119,3.36
5,0.0,0.0,0.56,0.0,0.0,0.44,0.4612,0.1388,3.32
6,0.0,0.0,0.67,0.0,0.0,0.33,0.5459,0.1657,3.29
7,0.0,0.0,0.78,0.0,0.0,0.22,0.6306,0.1926,3.27
8,0.0,0.0,0.89,0.0,0.0,0.11,0.7153,0.2195,3.26
9,0.0,0.0,1.00,0.0,0.0,0.00,0.8000,0.2464,3.25


In [ ]:
mu = expected_returns
cov = pln_returns[60:120][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10 = np.round(frontier.T,2)

In [ ]:
frontier_performance(frontier_5_10,mu,cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,IHYU.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.0,0.00,0.00,0.0,1.00,0.0300,0.0017,17.65
1,0.00,0.0,0.12,0.02,0.0,0.86,0.1234,0.0204,6.05
2,0.01,0.0,0.23,0.03,0.0,0.73,0.2092,0.0393,5.32
3,0.01,0.0,0.34,0.05,0.0,0.60,0.2949,0.0582,5.07
4,0.01,0.0,0.45,0.07,0.0,0.47,0.3805,0.0771,4.94
5,0.02,0.0,0.56,0.08,0.0,0.34,0.4663,0.0960,4.86
6,0.02,0.0,0.67,0.10,0.0,0.21,0.5520,0.1148,4.81
7,0.02,0.0,0.78,0.12,0.0,0.08,0.6377,0.1337,4.77
8,0.00,0.0,0.89,0.11,0.0,0.00,0.7208,0.1521,4.74
9,0.00,0.0,1.00,0.00,0.0,0.00,0.8000,0.1707,4.69


In [ ]:
shrinked_cov = shrinkedCovariance(pln_returns[60:120][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10_sh = np.round(frontier.T,2)

In [ ]:
frontier_performance(frontier_5_10_sh,mu,shrinked_cov)

,ACWI,^TBSP,XAUPLN,MWIG40TR,IHYU.L,PLOPLN3M,Returns,Std. deviations,SR
0,0.0,0.0,0.00,0.0,0.0,1.00,0.0300,0.0017,17.65
1,0.0,0.0,0.11,0.0,0.0,0.89,0.1147,0.0199,5.76
2,0.0,0.0,0.22,0.0,0.0,0.78,0.1994,0.0385,5.18
3,0.0,0.0,0.34,0.0,0.0,0.66,0.2918,0.0588,4.96
4,0.0,0.0,0.45,0.0,0.0,0.55,0.3765,0.0775,4.86
5,0.0,0.0,0.56,0.0,0.0,0.44,0.4612,0.0961,4.80
6,0.0,0.0,0.67,0.0,0.0,0.33,0.5459,0.1147,4.76
7,0.0,0.0,0.78,0.0,0.0,0.22,0.6306,0.1334,4.73
8,0.0,0.0,0.89,0.0,0.0,0.11,0.7153,0.1520,4.71
9,0.0,0.0,1.00,0.0,0.0,0.00,0.8000,0.1707,4.69


In [ ]:
acwi_tbsc = pln_returns[['ISAC.L','^TBSP']]
acwi_tbsc.describe()
np.sqrt(acwi_tbsc.std() ** 2 * 12) 

#acwi_tbsc.corr()

KeyError: "['ISAC.L'] not in index"

In [ ]:
means = expected_returns[['ISAC.L','^TBSP']]
cov = acwi_tbsc.cov()
number_of_scenarios = 1001
T = 120
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

In [ ]:
sim_means = sim_returns.mean(axis=1)
sim_means.mean(0) *12

sim_std = sim_returns.std(axis=1)

sim_std.mean(0) * np.sqrt(12)

array([0.11477853, 0.04748919])

In [ ]:
e = np.arange(0,1.1,0.1)
b = 1 - e
portfolios = np.vstack((e,b)).T


In [ ]:
scenarios = sim_returns + 1
scenarios = np.cumprod(scenarios,axis=1)
r = 10
T = np.arange(11,120,12)

glide_path = []

for t in T:
    qv = []
    for p in portfolios:
        lw = scenarios[:,t] @ p
        qv.append(np.percentile(lw, r))
    glide_path.append(np.argmax(qv))

print(glide_path)
np.take(portfolios,glide_path,0)

[2, 3, 3, 3, 4, 4, 5, 4, 5, 9]


array([[0.2, 0.8],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.4, 0.6],
       [0.4, 0.6],
       [0.5, 0.5],
       [0.4, 0.6],
       [0.5, 0.5],
       [0.9, 0.1]])

In [ ]:
class Simulation():
    def __init__(self, initial_value, returns, strategy) -> None:
        self.capital = initial_value
        self.returns = returns
        self.strategy = strategy 
    
   
    def step(self, current_value,weights,returns):
        current_assets_value = np.expand_dims(current_value,axis=0).T * weights
        next_assets_value =  np.round(current_assets_value * (1+returns),2)
        return next_assets_value.sum(1)
    
        
    def run(self, T):
        for t in range(T):
            self.capital = step(self.capital,self.strategy[t],self.returns[:,t])

    


In [ ]:

capital = np.full(10001,100)

T = np.arange(11,216,12)
risk = 10

glide_paths = {}

for t in T:
    perc = []
    for portfolio in range(len(portfolios)):
        strategy = np.tile(portfolios[portfolio],(216,1))
        simulator = Simulation(capital,sim_returns,strategy)
        simulator.run(t)
        perc.append(np.percentile(simulator.capital,risk))
    glide_paths[t] = np.argmax(perc)

ValueError: operands could not be broadcast together with shapes (10001,2) (1001,2) 

In [ ]:
np.take(portfolios,list(glide_paths.values()),0)

array([[0.2, 0.8],
       [0.3, 0.7],
       [0.3, 0.7],
       [0.4, 0.6],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.8, 0.2],
       [0.9, 0.1],
       [0.9, 0.1],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ]])

In [ ]:


capital = np.full(10001,100)

T = np.arange(11,120,12)
risk = 10


strategy = np.take(portfolios,list(glide_paths.values()),0)
strategy = np.repeat(strategy,12,axis=0)
simulator = Simulation(capital,sim_returns,strategy)
simulator.run(216)
perc = np.percentile(simulator.capital,risk)
gt = 50000

perc = perc/capital[0]

gt/perc


24831.148192292407

In [ ]:
ret = sim_returns
mu = ret.mean(1)
cov = np.zeros((1001,2,2))
for i in range(ret.shape[0]):
    cov[i]= np.cov(ret[i],rowvar=False)

In [ ]:
np.cov(ret[0],rowvar=False)

array([[9.16444603e-04, 3.73971440e-07],
       [3.73971440e-07, 1.81429710e-04]])

In [ ]:
portfolio = rp.Portfolio(pd.DataFrame(pln_returns[assets]))
#pln_returns[assets]

portfolio.mu = expected_returns
portfolio.cov = cov_matrix

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

In [ ]:
means = expected_returns[['ISAC.L','^TBSP']]
cov = acwi_tbsc.cov()
number_of_scenarios = 101
T = 60


sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

NameError: name 'expected_returns' is not defined

In [ ]:
start = '01.12.2006'
tbsp = loadStooqData('^TBSP',start,'m')
wibor3 = loadStooqData('PLOPLN3M',start,'m')
cpi = loadStooqData('cpimpl.m',start,'m')

In [ ]:
tbsp = np.log(tbsp/tbsp.shift(1))
wibor3 = wibor3/100/12
cpi = cpi/100

In [ ]:
tbsp.loc['2007-01-01':].mean()*12
#'2019-12-31'

0.03856047611547877

In [ ]:
wibor3.loc['2007-01-01':].mean() * 12

0.032938

In [ ]:
cpi.loc['2007-01-01':].mean() *12

0.036306122448979614

In [ ]:
print(tbsp.loc['2007-01-01':'2023-12-31'].mean()*12)
print(wibor3.loc['2007-01-01':'2019-12-31'].mean() * 12)
print(cpi.loc['2007-01-01':'2019-12-31'].mean() *12)

0.03856047611547877
0.03302307692307694
0.02115384615384617


In [ ]:
tbsp_cum = (1+tbsp.loc['2007-01-01':'2023-06-31']).cumprod()[-1]
cpi_cum = (1.00125+cpi.loc['2007-01-01':'2023-06-31']).cumprod()[-1]
wibor_cum = (1+wibor3.loc['2007-01-01':'2023-06-31']/12).cumprod()[-1]

In [ ]:
print(tbsp_cum)
print(cpi_cum)
print(wibor_cum)

1.8398585720934704
2.3014767048682354
1.045825968849171


In [ ]:
tbsp_cum/cpi_cum
#wibor_cum/cpi_cum

0.799425242150694

In [ ]:
assets = ['ACWI', '^TBSP', 'XAUPLN', 'MWIG40', 'PLOPLN3M']
means = expected_returns[assets]
cov = pln_returns[assets].cov()
number_of_scenarios = 5001
T = 60
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov, T))

sim_returns = np.array(sim_returns)

KeyError: "['MWIG40'] not in index"

array([[-0.01028103,  0.00209126,  0.01638971, ...,  0.0327413 ,
        -0.00698382,  0.0125469 ],
       [ 0.01227802,  0.01162168,  0.01933903, ...,  0.04234834,
        -0.01645275, -0.01815404],
       [ 0.01453904,  0.04649648, -0.00281429, ...,  0.00432735,
         0.02166505,  0.01155405],
       ...,
       [-0.00258086,  0.004308  ,  0.02123184, ..., -0.00962725,
         0.0009797 ,  0.00862168],
       [ 0.00055927, -0.00528157,  0.02755346, ...,  0.00301319,
         0.02506916,  0.02209101],
       [ 0.02792106,  0.02199077,  0.0059625 , ...,  0.02772814,
         0.01830803,  0.04519334]])

In [ ]:
model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'
points = 15
weights = np.zeros((number_of_scenarios,len(assets),points))

for p in range(number_of_scenarios):
    portfolio = rp.Portfolio(pd.DataFrame(sim_returns[p], columns=assets))
    portfolio.assets_stats(method_mu='hist', method_cov='hist')
    w = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
    weights[p,:] = w.to_numpy()


In [ ]:
w_means = np.round(weights.mean(0),2)
w_means_df = pd.DataFrame(w_means.T, columns=assets)

In [ ]:
w_means_df

,ACWI,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
0,0.00,0.00,0.00,0.00,0.01,0.98
1,0.05,0.07,0.03,0.03,0.02,0.79
2,0.08,0.12,0.05,0.06,0.04,0.65
3,0.12,0.17,0.08,0.08,0.05,0.51
4,0.15,0.18,0.10,0.11,0.06,0.39
5,0.18,0.18,0.13,0.14,0.08,0.30
6,0.22,0.16,0.15,0.16,0.08,0.23
7,0.25,0.14,0.18,0.19,0.07,0.17
8,0.27,0.11,0.21,0.21,0.07,0.12
9,0.28,0.09,0.24,0.24,0.06,0.09


In [ ]:
sim_returns.mean(1) * 12

array([[0.07970701, 0.03594519],
       [0.13821283, 0.04863202],
       [0.04961046, 0.02331223],
       [0.04724664, 0.01191947],
       [0.05141667, 0.04384099],
       [0.11690707, 0.05204425],
       [0.10728094, 0.03374413],
       [0.07218974, 0.03473295],
       [0.11561089, 0.01857777],
       [0.08200223, 0.0519025 ],
       [0.04469573, 0.04964911]])

In [ ]:
def turbulance(returns, mean, cov):
    diff = returns - mean
    result = diff.T @ np.linalg.inv(cov) @ diff 
    return result/len(mean)

assets = ['ACWI', '^TBSP', 'XAUPLN', 'MWIG40','PLOPLN3M']
returns = pln_returns[assets]
mean = returns.mean()
cov_matrix = returns.cov()

returns['Turbulance'] = returns.apply(turbulance, axis=1, args=(mean, cov_matrix))
returns['Reccesion'] = returns['Turbulance'] > 2

C:\Users\matsz\AppData\Local\Temp\ipykernel_7076\3968158932.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns['Turbulance'] = returns.apply(turbulance, axis=1, args=(mean, cov_matrix))
C:\Users\matsz\AppData\Local\Temp\ipykernel_7076\3968158932.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns['Reccesion'] = returns['Turbulance'] > 2


In [ ]:
turbulance_returns = returns[returns['Reccesion'] == True]
assets_performance(turbulance_returns)

C:\Users\matsz\AppData\Local\Temp\ipykernel_7076\1005095042.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  return returns.mean()/returns.std()


,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Turbulance,Reccesion
mean,0.009024,0.006089,0.024768,-0.020074,0.003992,3.334259,1.0
std,0.066701,0.029061,0.092326,0.127944,0.002077,1.446618,0.0
median,0.021268,0.010444,0.017806,-0.028532,0.005042,2.706081,1.0
skew,-0.333036,0.003765,0.569272,-0.527930,-0.780292,1.836923,0.0
kurtosis,-1.281388,-0.197833,0.059682,0.421363,-0.939744,4.076343,0.0
sharp,0.135289,0.209517,0.268268,-0.156895,1.922071,2.304865,inf


In [ ]:
turbulance_returns.corr()

,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Turbulance,Reccesion
ACWI,1.000000,0.174111,-0.206336,0.700007,-0.039364,-0.200193,NaN
^TBSP,0.174111,1.000000,0.183881,0.143445,0.148803,0.025295,NaN
XAUPLN,-0.206336,0.183881,1.000000,-0.244127,0.103738,-0.024600,NaN
MWIG40,0.700007,0.143445,-0.244127,1.000000,-0.262048,-0.492089,NaN
PLOPLN3M,-0.039364,0.148803,0.103738,-0.262048,1.000000,0.395843,NaN
Turbulance,-0.200193,0.025295,-0.024600,-0.492089,0.395843,1.000000,NaN
Reccesion,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
turbulente_mu = turbulance_returns[assets].mean()
turbulente_cov = turbulance_returns[assets].cov()

porfolio = rp.Portfolio(turbulance_returns[assets].dropna())
portfolio.mu = turbulente_mu
portfolio.cov = turbulente_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_turbulent = np.round(frontier.T,2)
frontier_performance(frontier_turbulent, mu, cov, 0.004*12)

,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.00,0.00,0.0,1.00,0.0479,0.0072,-0.01
1,0.09,0.00,0.12,0.0,0.78,0.0828,0.0405,0.86
2,0.16,0.01,0.23,0.0,0.59,0.1147,0.0760,0.88
3,0.24,0.02,0.34,0.0,0.40,0.1477,0.1122,0.89
4,0.31,0.03,0.44,0.0,0.22,0.1771,0.1451,0.89
5,0.38,0.04,0.55,0.0,0.03,0.2090,0.1808,0.89
6,0.32,0.00,0.68,0.0,0.00,0.2368,0.2148,0.88
7,0.20,0.00,0.80,0.0,0.00,0.2594,0.2504,0.84
8,0.10,0.00,0.90,0.0,0.00,0.2783,0.2840,0.81
9,0.00,0.00,1.00,0.0,0.00,0.2972,0.3198,0.78


In [ ]:
non_turbulance_returns = returns[returns['Reccesion'] == False]
assets_performance(non_turbulance_returns)

C:\Users\matsz\AppData\Local\Temp\ipykernel_7076\1005095042.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return returns.mean()/returns.std()


,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Turbulance,Reccesion
mean,0.008341,0.002880,0.004373,0.005210,0.002498,0.694747,0.0
std,0.029806,0.009011,0.046709,0.045706,0.001548,0.461101,0.0
median,0.008651,0.003299,-0.000034,0.007032,0.002158,0.598745,0.0
skew,-0.123872,-0.433985,0.132541,-0.028483,0.574045,0.975154,0.0
kurtosis,0.289099,0.786131,0.521170,0.561621,-0.506861,0.158551,0.0
sharp,0.279846,0.319645,0.093620,0.113989,1.613661,1.506713,NaN


In [ ]:
non_turbulance_returns.corr()

,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Turbulance,Reccesion
ACWI,1.000000,0.025267,0.110452,0.284730,-0.213714,-0.209200,NaN
^TBSP,0.025267,1.000000,-0.000342,0.198929,0.108377,-0.170146,NaN
XAUPLN,0.110452,-0.000342,1.000000,-0.262842,-0.051331,0.081325,NaN
MWIG40,0.284730,0.198929,-0.262842,1.000000,-0.074714,-0.123575,NaN
PLOPLN3M,-0.213714,0.108377,-0.051331,-0.074714,1.000000,0.254948,NaN
Turbulance,-0.209200,-0.170146,0.081325,-0.123575,0.254948,1.000000,NaN
Reccesion,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
non_turbulance_mu = non_turbulance_returns[assets].mean()
non_turbulance_cov = non_turbulance_returns[assets].cov()

porfolio = rp.Portfolio(non_turbulance_returns[assets].dropna())
portfolio.mu = non_turbulance_mu
portfolio.cov = non_turbulance_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_non_turbulent = np.round(frontier.T,2)
frontier_performance(frontier_non_turbulent, mu, cov, 0.002498*12)

,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Returns,Std. deviations,SR
0,0.01,0.01,0.00,0.00,0.98,0.0488,0.0076,2.48
1,0.15,0.08,0.01,0.00,0.75,0.0610,0.0370,0.84
2,0.26,0.14,0.02,0.00,0.59,0.0726,0.0636,0.67
3,0.36,0.19,0.02,0.00,0.43,0.0794,0.0878,0.56
4,0.46,0.24,0.03,0.00,0.27,0.0892,0.1120,0.53
5,0.56,0.30,0.03,0.00,0.10,0.0963,0.1368,0.48
6,0.66,0.29,0.04,0.01,0.00,0.1021,0.1616,0.45
7,0.77,0.18,0.04,0.01,0.00,0.1060,0.1831,0.42
8,0.88,0.07,0.04,0.01,0.00,0.1099,0.2056,0.39
9,1.00,0.00,0.00,0.00,0.00,0.1083,0.2311,0.34


In [ ]:
mu = 0.2*turbulente_mu + 0.8 * non_turbulance_mu
cov = 0.2*turbulente_cov + 0.8 * non_turbulance_cov

porfolio = rp.Portfolio(returns[assets].dropna())
portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_non_turbulent = np.round(frontier.T,2)
frontier_performance(frontier_non_turbulent, mu, cov, 0.002498*12)

,ACWI,^TBSP,XAUPLN,MWIG40,PLOPLN3M,Returns,Std. deviations,SR
0,0.00,0.00,0.00,0.0,0.99,0.0332,0.0057,0.57
1,0.12,0.03,0.06,0.0,0.79,0.0461,0.0205,0.79
2,0.21,0.06,0.10,0.0,0.63,0.0552,0.0351,0.72
3,0.30,0.09,0.14,0.0,0.47,0.0643,0.0499,0.69
4,0.39,0.12,0.18,0.0,0.31,0.0734,0.0648,0.67
5,0.47,0.15,0.23,0.0,0.16,0.0829,0.0797,0.66
6,0.56,0.17,0.27,0.0,0.00,0.0915,0.0944,0.65
7,0.66,0.02,0.32,0.0,0.00,0.1004,0.1094,0.64
8,0.89,0.00,0.11,0.0,0.00,0.1017,0.1241,0.58
9,1.00,0.00,0.00,0.0,0.00,0.1017,0.1386,0.52
